In [1]:
import warnings
warnings.filterwarnings('ignore')
import random 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h3
import json

In [2]:
resolution = 7
hex_col = 'hex'+str(resolution)
sample_size = 200

# Charger type

In [3]:
df = pd.read_csv('./Bristol charging points.csv')
df = df.groupby('connector2Type').count()
df['connector2Type'] = df.index
df = df[['connector2Type', 'chargeDeviceID']]
df.columns = ['connector2Type','prob']
df['prob'] = df['prob'] / df['prob'].sum()


In [4]:
df_ct =  pd.DataFrame( np.random.choice(a =df['connector2Type'],size=sample_size , p=df['prob']))
df_ct.columns = ['connector2Type']
df_ct

,connector2Type
0,Type 2 Mennekes (IEC62196)
1,CCS Type 2 Combo (IEC62196)
2,JEVS G105 (CHAdeMO) DC
3,Type 2 Mennekes (IEC62196)
4,Type 2 Mennekes (IEC62196)
...,...
195,Type 2 Mennekes (IEC62196)
196,CCS Type 2 Combo (IEC62196)
197,Type 2 Mennekes (IEC62196)
198,JEVS G105 (CHAdeMO) DC


In [6]:
df_ct.to_csv('_connector2Type.csv')

# Map

In [7]:
df_map = pd.read_csv('./data_simulation_maps.csv')
df_map['probility'] =(df_map['mean_car'] / df_map['mean_car'].sum() )  
df_map.head()
maps_count = len(df_map)

In [8]:
random_numebrs2 =  np.random.choice(a =np.arange(0, maps_count),size= sample_size, p=df_map['probility'])

df_map.index.rename('ind', inplace=True)
df_map['index'] = df_map.index
p = df_map['index'].isin(random_numebrs2)  
a = df_map[p].copy()
df_map_f = a[['Latitude','Longitude']]
df_map_f

,Latitude,Longitude
ind,,
246,51.477768,-2.721353
288,51.468992,-2.567747
896,51.478591,-2.609711
1735,51.436433,-2.605611
2386,51.477908,-2.701398
...,...,...
45213,51.480365,-2.535099
45256,51.344069,-2.969216
45628,51.414525,-2.629953


In [10]:
df_map_f.to_csv('_map.csv')

# Home

In [11]:
df_h = pd.read_csv('./Bristol_Postcodes.csv')
df_home = df_h[['Postcode' , 'In Use?' , 'Latitude', 'Longitude']]
df_home = df_home[ df_home['In Use?'] == 'Yes' ]
df_m = df_map[[hex_col,'mean_car']]
df_home[hex_col] = df_home.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,resolution),1)
df_home = df_home.merge(df_m, how='right', on=hex_col) 
df_home['probility'] =((df_home['mean_car']) / (df_home['mean_car'].sum() ))  
df_home.head()

,Postcode,In Use?,Latitude,Longitude,hex7,mean_car,probility
0,BS1 1AD,Yes,51.458457,-2.574165,871958390ffffff,877.440725,8.215961e-08
1,BS1 1DD,Yes,51.459757,-2.574944,871958390ffffff,877.440725,8.215961e-08
2,BS1 9DA,Yes,51.458457,-2.574165,871958390ffffff,877.440725,8.215961e-08
3,BS1 9DB,Yes,51.458457,-2.574165,871958390ffffff,877.440725,8.215961e-08
4,BS1 9DL,Yes,51.458457,-2.574165,871958390ffffff,877.440725,8.215961e-08


In [12]:
home_count = len(df_home)

In [13]:
random_numebrs3 =  np.random.choice(a =np.arange(0, home_count),size= sample_size, p=df_home['probility'])
df_home['index'] = df_home.index
p = df_home['index'].isin(random_numebrs3)  
a = df_home[p].copy()
df_home = a[['Latitude','Longitude']]
df_home

,Latitude,Longitude
96681,51.458764,-2.575336
132207,51.457543,-2.580702
152553,51.461915,-2.592979
237084,51.485791,-2.604666
253840,51.427458,-2.567721
...,...,...
17486532,51.487108,-2.617645
17532029,51.441518,-2.560831
17635686,51.458611,-2.564639
17684503,51.462016,-2.601129


In [14]:
df_home.to_csv('_home.csv')

# Combining

In [15]:
df = df.sample(frac=1)
df_home = df_home.sample(frac=1)
df_map = df_map_f.sample(frac=1)

df = pd.DataFrame( {
                'connector2Type': df_ct['connector2Type'].to_numpy()
            , 'home_lat': df_home['Latitude'].to_numpy()
            , 'home_long': df_home['Longitude'].to_numpy()
            , 'car_lat': df_map['Latitude'].to_numpy()
            , 'car_long': df_map['Longitude'].to_numpy()
            })

In [16]:
df.to_csv('_combined_samples.csv')

# Json

In [17]:
result = []
for _, row in df.iterrows():
    result.append({
        
            'home': {
                'Latitude': row['home_lat'],
                'Longitude': row['home_long'],
            },
            'car': {
                'Latitude': row['car_lat'],
                'Longitude': row['car_long'],
                'connector2Type': row['connector2Type']
            }
       
    })

# Convert the list of dictionaries to JSON
json_output = json.dumps(result, indent=4)

# Save to a file (you can choose your desired filename)
with open('user_data_all_rows.json', 'w') as json_file:
    json_file.write(json_output)

print("Data saved to user_data_all_rows.json")


Data saved to user_data_all_rows.json
